In [1]:
# Sparse solution for Linear Equation
## sparse solution: one with few nonzeros

In [3]:
%reset -f
import gurobipy as gp
from gurobipy import GRB
import numpy as np

In [4]:
# Parameters
m = 50  # Number of equations
n = 500  # Number of variables

In [5]:
# 1. Generate A_{m*n} with entries 0 or 1 (probability 0.5)
A = np.random.randint(2, size=(m, n))

In [6]:
# 2. Sparse vector z
z = np.zeros(n)
z[:m//2] = np.random.uniform(-1, 1, size=m//2) # 1st m/2 entries with random val in [-1,1]

In [11]:
# 3. b = A * z
b = A @ z

In [13]:
# 4. Set up the Gurobi model
model = gp.Model("1_norm_minimization")

# Add variables x_j, with bounds -1 <= x_j <= 1
x = model.addVars(n, lb=-1, ub=1, name="x") # create n vars; compents of \vec{x}

# Add auxiliary variables for absolute values of x_j
u = model.addVars(n, lb=0, name="u") # create n aux vars u; abs val of x_j

Restricted license - for non-production use only - expires 2025-11-24


In [15]:
# Objective: Minimize the sum of absolute values |x_j|
model.setObjective(gp.quicksum(u[j] for j in range(n)), GRB.MINIMIZE) # sum all u_j, minimize hence 1-norm

# Constraints: A @ x == b
for i in range(m):
    model.addConstr(gp.quicksum(A[i, j] * x[j] for j in range(n)) == b[i], name=f"eq_{i}") # add m constraints; sum up A[i,j] 
                                                                                           # enforce j to correspond each entries b[i]

# Constraints: u[j] >= |x_j|
for j in range(n):
    model.addConstr(u[j] >= x[j], name=f"u_ge_x_{j}")
    model.addConstr(u[j] >= -x[j], name=f"u_ge_neg_x_{j}") # force u_j to be abs of x_j for 1-norm

In [16]:
# Optimize the model
model.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[x86] - Darwin 24.0.0 24A335)

CPU model: Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1050 rows, 1000 columns and 14504 nonzeros
Model fingerprint: 0x9286a5a6
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e-01, 6e+00]
Presolve time: 0.08s
Presolved: 1050 rows, 1000 columns, 14504 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   9.921244e+03   0.000000e+00      0s
    2323    1.0910936e+01   0.000000e+00   0.000000e+00      0s

Solved in 2323 iterations and 0.24 seconds (0.18 work units)
Optimal objective  1.091093636e+01


In [17]:
# 5. Results
sparse_solution = np.array([x[j].X for j in range(n)])
non_zero_count = np.sum(np.abs(sparse_solution) > 1e-6)

print(f"Number of non-zeros in the solution: {non_zero_count}")
print(f"Solution x: {sparse_solution}")

Number of non-zeros in the solution: 50
Solution x: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.11650806 -0.39645576  0.06140898  0.24842498
  0.          0.51391748  0.          0.31543265  0.         -0.01040729
  0.02205386  0.60967559  0.          0.04876255  0.47258601 -0.66432023
  0.          0.         -0.00859574  0.30013605 -0.20513332  0.
  0.          0.          0.13852673  0.          0.          0.35214109
  0.          0.          0.          0.          0.          0.
  0.          0.          0.30439753  0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.34175362  0.          0.
  0.          0.          0.          0.          0.07026172 -0.16696254
  0.          0.          0.          0.          0.          0